## Harjoitustyö - Miika Toukola, Datan esikäsittely TTC8030-3007 kevät 2023

## Vaihe 1 - some info on recent trains in a DataFrame

#### Tehtävä 1. Hae Digitrafficin REST-rajapintapalvelusta (JSON-formaatissa) aktiiviset junat pääteosoitteella `/live-trains` Helsingin asemalta, joiden hakuparametrit ovat seuraavat:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [11]:
import requests

junaturl =  "https://rata.digitraffic.fi/api/v1/live-trains"
asema = "/station/HKI"
parametrit = "?arrived_trains=5&arriving_trains=5&departed_trains=5&departing_trains=5&include_nonstopping=false"

#/live-trains/station/HKI?arrived_trains=10&departed_trains=10&include_nonstopping=false

req = requests.get(url=junaturl+asema+parametrit)

#req.status_code

junadata_json = req.json()

print(len(junadata_json))

20


#### Tehtävä 2. Tämän jälkeen lue kyseinen data Pandasin dataframeen ilman muokkaamista, jolloin sarakkeiden otsikot ovat seuraavanlaiset:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version` 

In [14]:
import pandas as pd

junadata = pd.DataFrame.from_dict(junadata_json)
junadata.head()

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,2,2023-03-04,10,vr,S,Long-distance,,True,False,285038871797,REGULAR,2022-11-03T06:29:25.000Z,"[{'stationShortCode': 'JNS', 'stationUICCode':..."
1,42,2023-03-04,10,vr,IC,Long-distance,,True,False,285038876442,REGULAR,2022-11-03T06:29:25.000Z,"[{'stationShortCode': 'VS', 'stationUICCode': ..."
2,43,2023-03-04,10,vr,IC,Long-distance,,False,False,285038855087,REGULAR,2022-08-15T06:10:16.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
3,946,2023-03-04,10,vr,IC,Long-distance,,False,False,285038876442,REGULAR,2022-12-15T06:04:43.000Z,"[{'stationShortCode': 'KUT', 'stationUICCode':..."
4,8063,2023-03-04,10,vr,HL,Commuter,A,False,False,285030844986,REGULAR,2022-12-15T06:04:43.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


#### Tehtävä 3. Poista kaikki muut sarakkeet paitsi `timeTableRows` ja `trainNumber`

In [18]:
junadata.drop(junadata.iloc[:, 1:12], inplace=True, axis = 1)
junadata.head()

,trainNumber,timeTableRows
0,2,"[{'stationShortCode': 'JNS', 'stationUICCode':..."
1,42,"[{'stationShortCode': 'VS', 'stationUICCode': ..."
2,43,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
3,946,"[{'stationShortCode': 'KUT', 'stationUICCode':..."
4,8063,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


#### Tehtävä 4. Tämän jälkeen pura `timeTableRows` sarakkeen arvot jokaisesta junasta erikseen (sarakkeen arvo jokaiselle junalle sisältää useampia JSON-objekteja) ja lisää dataan mukaan junan tiedot yksilöivä `trainNumber`

In [85]:
#punadata = pd.DataFrame(columns= ['trainNumber']+list(junadata.iloc[0,1][0].keys()) )

def goup (n):
    goupdf = pd.DataFrame.from_dict(junadata.iloc[n,1])
    goupdf['trainNumber'] = junadata.iloc[n,0]    
    #print(goupdf)
    #punadata = pd.concat([punadata,goupdf])
    return goupdf
    
punadata = goup(0)

for i in range(1,len(junadata)):
    punadata = pd.concat([punadata,goup(i)])
    #print(goup(i).head())
    #print(goup(i).columns.values)

print('The length of the new DataFrame is {0} rows.'.format(len(punadata.index)))

punadata.reset_index(inplace=True,drop=True)

#print(punadata.columns.values)

punadata.head()

The length of the new DataFrame is 1216 rows.


,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,differenceInMinutes,causes,trainNumber,trainReady
0,JNS,460,FI,DEPARTURE,True,True,1,False,2023-03-04T02:58:00.000Z,2023-03-04T02:58:00.000Z,LIIKE_AUTOMATIC,2023-03-04T02:58:00.000Z,0.0,[],2,"{'source': 'KUPLA', 'accepted': True, 'timesta..."
1,PLT,1070,FI,ARRIVAL,False,NaN,,False,2023-03-04T02:58:42.000Z,2023-03-04T02:58:42.000Z,LIIKE_AUTOMATIC,NaN,0.0,[],2,NaN
2,PLT,1070,FI,DEPARTURE,False,NaN,,False,2023-03-04T02:58:42.000Z,2023-03-04T02:58:42.000Z,LIIKE_AUTOMATIC,NaN,0.0,[],2,NaN
3,SUL,1071,FI,ARRIVAL,False,NaN,,False,2023-03-04T02:59:32.000Z,2023-03-04T02:59:32.000Z,LIIKE_AUTOMATIC,NaN,0.0,[],2,NaN
4,SUL,1071,FI,DEPARTURE,False,NaN,,False,2023-03-04T02:59:32.000Z,2023-03-04T02:59:32.000Z,LIIKE_AUTOMATIC,NaN,0.0,[],2,NaN


#### Tehtävä 5. Pudota sitten seuraavat sarakkeet datasta: `causes`, `estimateSource`, `liveEstimateTime`, `differenceInMinutes`, `commercialStop` ja `trainReady`

In [86]:
punadata.drop(['causes','estimateSource','liveEstimateTime','differenceInMinutes','commercialStop','trainReady'], inplace=True, axis = 1)
#print('The remaining DataFrame has a length of {0} rows.'.format(len(punadata.index)))
punadata.head()

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainNumber
0,JNS,460,FI,DEPARTURE,True,1,False,2023-03-04T02:58:00.000Z,2023-03-04T02:58:00.000Z,2
1,PLT,1070,FI,ARRIVAL,False,,False,2023-03-04T02:58:42.000Z,NaN,2
2,PLT,1070,FI,DEPARTURE,False,,False,2023-03-04T02:58:42.000Z,NaN,2
3,SUL,1071,FI,ARRIVAL,False,,False,2023-03-04T02:59:32.000Z,NaN,2
4,SUL,1071,FI,DEPARTURE,False,,False,2023-03-04T02:59:32.000Z,NaN,2


#### Tehtävä 6. Täytä jokaisen rivin sarakkeen `actualTime` puuttuvat arvot `scheduledTime` -sarakkeen arvolla.

In [91]:
punadata['actualTime'] = punadata['actualTime'].fillna(punadata['scheduledTime'])
punadata.head()

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainNumber
0,JNS,460,FI,DEPARTURE,True,1,False,2023-03-04T02:58:00.000Z,2023-03-04T02:58:00.000Z,2
1,PLT,1070,FI,ARRIVAL,False,,False,2023-03-04T02:58:42.000Z,2023-03-04T02:58:42.000Z,2
2,PLT,1070,FI,DEPARTURE,False,,False,2023-03-04T02:58:42.000Z,2023-03-04T02:58:42.000Z,2
3,SUL,1071,FI,ARRIVAL,False,,False,2023-03-04T02:59:32.000Z,2023-03-04T02:59:32.000Z,2
4,SUL,1071,FI,DEPARTURE,False,,False,2023-03-04T02:59:32.000Z,2023-03-04T02:59:32.000Z,2


## Vaihe 2 - adding more data from a file into the DataFrame

#### Tehtävä 7. Pura tämän kansion sisällä olevan `data.html` -tiedoston `table`-elementin solujen arvot käyttäen regex-syntaksia ja yhdistä ne vaiheessa 1 luotuun dataframeen. Huom: tehtävässä tulee käyttää vain regexiä (Esim. Pandasin read_html()-metodin käyttö on kielletty)!

In [6]:
import re



## Vaihe 3

#### Tehtävä 8. Tee SQLite tietokanta ja lisää sinne vaiheen 2 tuloksena saatu dataframe.


In [7]:
#